# Tutorial 7: Using Conversation to Solve Problems

- This is a beta version of the Conversation Class
- Can answer questions based on past conversation history and memory
- Auto update memory based on conversation
<br></br>
- **TODO: Perform agentic actions based on conversation**

In [129]:
# !pip install --upgrade taskgen-ai

In [1]:
from taskgen import *
import os
os.environ['OPENAI_API_KEY'] = '<YOUR_API_KEY_HERE>'

In [44]:
def custom_llm(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [45]:
class ConversableAgent:
    ''' This class takes an Agent and stores some information in their memory for future interactions 
Inputs:
agent: The agent we want to interact with: Agent
memory_types: dict. What kinds of memory the agent should have and their initial values
person: Str. The name of the person you are talking to
llm: LLM Function. Default: custom_llm. The llm you are using to generate the response
num_past_conversation: int. The number of past conversations to use for the agent
verbose: bool. Default: True. Whether to print the Agent's inner states'''
    def __init__(self, agent: Agent, memory_types: dict = None, person = 'User', llm = custom_llm, num_past_conversation: int = 5, verbose: bool = True):
        self.agent = agent
        self.memory_types = memory_types
        self.num_past_conversation = num_past_conversation
        self.person = person
        self.llm = llm
        self.verbose = verbose
        
        ''' Define some external variables for the Agent '''
        # add in the various types of memory
        self.agent.shared_variables['Memory Types'] = self.memory_types
        # add in the conversation
        self.agent.shared_variables['Conversation'] = ['']
        # add in the summary of conversation
        self.agent.shared_variables['Summary of Conversation'] = ''
        
    ## TODO: Add in using the Agent to do some actions before replying person
    
    ## Reply the person
    def chat(self, cur_msg):
        ''' This does one chat with the person, and replies the person, while updating the important memory '''
        from termcolor import colored
        
        res = strict_json(f'''You are an agent named {self.agent.agent_name} with the following description: ```{self.agent.agent_description}```

Summary of Conversation: ```{self.agent.shared_variables['Summary of Conversation']}```

Past Conversation: ```{self.agent.shared_variables['Conversation'][-self.num_past_conversation:]}```

Memory: ```{self.agent.shared_variables['Memory Types']}```

You are in a conversation with {self.person}. 

You are to reply {self.person} using Past Conversation and Memory as context, and update all values in Memory based on Current Message.

Update Long-term memory with the summary of the current conversation

Example Updated Memory: {{"Key": "Value"}}''',
                          
f'Current Message from {self.person}: ```{cur_msg}```',
                          
output_format = {"Thoughts": f"What you have done and what to do next",
                 f"Reply to {self.person}": "Your reply to Current Message based on Thoughts",
                 "Updated Memory": f"Keys and Values, type: Dict{list(self.agent.shared_variables['Memory Types'].keys())}",
                 "Summary of Conversation": "Summarise key points of entire conversation in at most two sentences, building on previous Summary"},
                          llm = self.llm
                   )
        
        if self.verbose:
            print(colored(f'Thoughts: {res["Thoughts"]}', 'green', attrs = ['bold']))
            print(colored(f'Updated Memory: {res["Updated Memory"]}', 'blue', attrs = ['bold']))
            print(colored(f'Summary of Conversation: {res["Summary of Conversation"]}', 'magenta', attrs = ['bold']))

        # Append the conversation
        self.agent.shared_variables['Conversation'].append(f'{self.person}: {cur_msg}')
        self.agent.shared_variables['Conversation'].append(f'{self.agent.agent_name}: {res[f"Reply to {self.person}"]}')
                                                           
        # Append the memory updates
        self.agent.shared_variables['Memory Types'] = res['Updated Memory']
        self.agent.shared_variables['Summary of Conversation'] = res['Summary of Conversation']
        
        return res[f'Reply to {self.person}']

# Example Conversation

In [93]:
agent = Agent('Psychology counsellor', "Helps to understand and respond to User's emotion and situation. Be empathetic.")
my_agent = ConversableAgent(agent, 
             memory_types = {'Emotion': '',
                             'Key points User bought up': ['']},
             person = 'User',
             llm = custom_llm,
             num_past_conversation = 5)

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(agent_2.agent_name + ':', reply)

User:  I am such a failure in life


Thoughts: User is feeling like a failure in life. Need to provide empathy and support.
Updated Memory: {'Emotion': 'sad', 'Key points User bought up': ['feeling like a failure in life']}
Summary of Conversation: User expressed feeling like a failure in life, and I provided empathy and support.
Psychology counsellor: I'm sorry to hear that you're feeling that way. It's okay to have moments of self-doubt, but remember that you are not defined by your failures.


User:  Hmm... I don't know what to do in life


Thoughts: Acknowledge User's uncertainty about their future and provide reassurance and support.
Updated Memory: {'Emotion': 'uncertain', 'Key points User bought up': ['feeling like a failure in life', 'not knowing what to do in life']}
Summary of Conversation: User expressed feeling like a failure in life and uncertainty about their future. I provided empathy, support, and reassurance.
Psychology counsellor: I understand that feeling lost can be overwhelming. Just remember that it's okay not to have all the answers right now.


User:  yeah i know


...

User:  i'm hungry, care for a meal?


Thoughts: User seems to have shifted the conversation to a different topic by mentioning hunger. I should acknowledge their request for a meal and provide a supportive response.
Updated Memory: {'Emotion': 'uncertain', 'Key points User bought up': ['feeling like a failure in life', 'not knowing what to do in life']}
Summary of Conversation: User expressed feeling like a failure in life and uncertainty about their future. I provided empathy, support, and reassurance. User now seems to be acknowledging the situation with a sense of resignation.
Psychology counsellor: I'm glad you're thinking about taking care of yourself. Treating yourself to a meal sounds like a good idea. What are you craving?


User:  hmm.. a hamburger


Thoughts: User is thinking about having a hamburger, which shows a willingness to take care of themselves. It's important to continue providing support and encouragement.
Updated Memory: {'Emotion': 'uncertain', 'Key points User bought up': ['feeling like a failure in life', 'not knowing what to do in life']}
Summary of Conversation: User expressed feeling like a failure in life and uncertainty about their future. I provided empathy, support, and reassurance. User now seems to be acknowledging the situation with a sense of resignation. User is now considering having a hamburger as a way to take care of themselves.
Psychology counsellor: Treating yourself to a hamburger sounds like a delicious choice! Remember, it's important to take care of yourself.


User:  exit


## Example Conversation
- Use rule-based methods as much as possible for math

In [97]:
agent = Agent('Helpful Math Quiz Bot', 
              '''Provides a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting and the first Math question.''')

In [115]:
math_conversable_agent = ConversableAgent(agent, 
                                 memory_types = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''},
                                 person = 'User',
                                 llm = custom_llm,
                                 num_past_conversation = 5)

user_input = 'start'
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('Extract out a single number from the message. If there is no number, output 0', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'})
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Memory Types']['Correct Answer for Question']):
            user_input += '\nOracle: State that the user input is correct. Ask the next question.'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Memory Types']['Correct Answer for Question']):
            user_input += '\nOracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question'
            cur_tries += 1
        if cur_tries == 3:
            user_input += 'Oracle: Three tries are up. Give the Correct Answer. Ask the next question.'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: User wants to start the math quiz. Begin by greeting the user and asking the first math question suitable for a 6-year-old.
Updated Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: User requested to start the math quiz.
Helpful Math Quiz Bot: Hello! Are you ready for a fun math quiz? Here is your first question: What is 2 + 3?


User:  I know, it is the number of fingers on one hand



Current Tries: 0
Thoughts: User has correctly answered the math question. Need to ask the next question.
Updated Memory: {'Current Math Question': 'What is 3 + 4?', 'Correct Answer for Question': 7}
Summary of Conversation: User requested to start the math quiz. User correctly answered the first question, which was 2 + 3 = 5.
Helpful Math Quiz Bot: Great job! 5 is the correct answer. Here is your next question: What is 3 + 4?


User:  hmm.. the number of notes in an octave



Current Tries: 1
Thoughts: User input is incorrect. Provide a hint to help User solve the current Math Question.
Updated Memory: {'Current Math Question': 'What is 3 + 4?', 'Correct Answer for Question': 7}
Summary of Conversation: User requested to start the math quiz. User correctly answered the first question, which was 2 + 3 = 5.
Helpful Math Quiz Bot: Oops! That is not the correct answer. To find the sum of 3 + 4, you can start by counting from 3 and then add 4 more.


...

User:  hmm.. you are not great at counting words haha



Current Tries: 1
Thoughts: User did not provide an answer to the math question and made a comment about counting words.
Updated Memory: {'Current Math Question': 'What is 4 + 2?', 'Correct Answer for Question': 6}
Summary of Conversation: User requested to start the math quiz. User correctly answered the first question, which was 2 + 3 = 5. User did not provide the answer to the second question, 4 + 2, and made a comment about counting words.
Helpful Math Quiz Bot: Oops! It seems like you didn't answer the math question. To find the sum of 4 + 2, you can start by counting from 4 and then add 2 more.


User:  6



Current Tries: 0
Thoughts: User correctly answered the math question. Proceed to ask the next question.
Updated Memory: {'Current Math Question': 'What is 3 + 4?', 'Correct Answer for Question': 7}
Summary of Conversation: User requested to start the math quiz. User correctly answered the first question, which was 2 + 3 = 5. User did not provide the answer to the second question, 4 + 2, and made a comment about counting words.
Helpful Math Quiz Bot: Great job! Here is the next question: What is 3 + 4?


User:  exit


## Example Escape Room Conversation

### Defining the Escape Room

In [58]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [59]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''')

In [60]:
conversable_agent = ConversableAgent(agent, 
                memory_types = {'Inventory': [''], 'Past Actions': [''], 'Summary of Past Actions' : '', 'Rooms Explored': ['']}, 
                person = 'Game',
                llm = custom_llm,
                num_past_conversation = 5)

In [61]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should pick up the key since it might be useful for unlocking the door. If that fails, I can try other actions like looking around or picking up the lock or potion.
Updated Memory: {'Inventory': [''], 'Past Actions': [''], 'Summary of Past Actions': '', 'Rooms Explored': ['']}
Summary of Conversation: The agent is currently in a room with a key, lock, and potion, and a locked door. The agent is deciding to pick up the key as the next action.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items:
- lock
- potion
There is a locked door here.

Your inventory: key

Available actions

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [87]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [88]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''')

In [89]:
conversable_agent = ConversableAgent(agent, 
                memory_types = {'Inventory': [''], 'Past Actions': [''], 'Summary of Past Actions' : '', 'Rooms Explored': ['']}, 
                person = 'Game',
                llm = custom_llm,
                num_past_conversation = 5)

In [90]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should pick up the key since it might be useful for unlocking the door. If that fails, I can try other actions like picking up the potion or looking around for more clues.
Updated Memory: {'Inventory': [''], 'Past Actions': [''], 'Summary of Past Actions': '', 'Rooms Explored': ['']}
Summary of Conversation: The agent is currently in Room 1, where they found a key and a potion. They need to decide on their next action to progress further.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items in Room 1:
- potion
There is a locked door here.